In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//span[@itemprop="name"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                subtitle = html.xpath('//div[@class="ti-type"]/h2[@class="cad_header"]/span[@class="ti-value"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                list_vehicle = html.xpath('//div[@id="country3-head"]/following-sibling::div[@class="pro_tab_container"]/table/tr/td/text()')
                for vehicle in list_vehicle:
                    vehicle = vehicle.strip()
                    if vehicle != '':
                        year = int(vehicle.split(' ')[0])
                        make_model_engine = ' '.join(vehicle.split(' ')[1:])
                        if make_model_engine not in dict_vehicle:
                            dict_vehicle[make_model_engine] = [year]
                        else:
                            dict_vehicle[make_model_engine].append(year)

                        dict_vehicle[make_model_engine] = sorted(list(set(dict_vehicle[make_model_engine])))

                # = = = = = = = = = = = = = = =
                                
                list_vehicle = html.xpath('//div[@id="country3-head"]/following-sibling::div[@class="pro_tab_container"]/table/tr/td/div[@class="accordion_container"]/div[@class="accordion_body"]/ul/li/text()')
                for vehicle in list_vehicle:
                    vehicle = vehicle.strip()
                    if vehicle != '' and vehicle != '...and more':
                        year = int(vehicle.split(' ')[0])
                        make_model_engine = ' '.join(vehicle.split(' ')[1:])
                        if make_model_engine not in dict_vehicle:
                            dict_vehicle[make_model_engine] = [year]
                        else:
                            dict_vehicle[make_model_engine].append(year)

                        dict_vehicle[make_model_engine] = sorted(list(set(dict_vehicle[make_model_engine])))

                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make_model_engine in dict_vehicle:
                    year_min = min(dict_vehicle[make_model_engine])
                    year_max = max(dict_vehicle[make_model_engine])
                    if year_max == year_min:
                        list_vehicle.append(make_model_engine + ' ' + str(year_min))
                    else:
                        list_vehicle.append(make_model_engine + ' ' + str(year_min) + '-' + str(year_max))

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//img[@itemprop="image"]/@src')[0].strip()

                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//div[@id="country2-head"]/following-sibling::div[@class="pro_tab_container"]/table/tr')

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Title': title,
                                         'Subtitle': subtitle,
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src}])

                for tr in list_tr:
                    name = tr.xpath('./td[1]/text()')[0].strip()
                    if name == '':
                        continue
                    df_temp.loc[0, name] = re.sub(';+', ';', ';'.join([text.strip() for text in tr.xpath('./td[2]/descendant-or-self::*/text()')]).strip())
                    df_temp.loc[0, name] = re.sub('^;', '', df_temp.loc[0, name])
                    df_temp.loc[0, name] = re.sub(';$', '', df_temp.loc[0, name])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：360

[ok] - https://www.turbochargerpros.com/buynow/40-39135_gg
[尝试次数：1] - [剩余数量：340] - [当前时间：10:33:50]

[ok] - https://www.turbochargerpros.com/buynow/40-30108_gg
[尝试次数：2] - [剩余数量：339] - [当前时间：10:33:51]

[ok] - https://www.turbochargerpros.com/buynow/40-30354_gg
[尝试次数：5] - [剩余数量：338] - [当前时间：10:33:52]

[ok] - https://www.turbochargerpros.com/buynow/40-35705_gg
[尝试次数：1] - [剩余数量：337] - [当前时间：10:33:52]

[ok] - https://www.turbochargerpros.com/buynow/40-30281_gg
[尝试次数：6] - [剩余数量：336] - [当前时间：10:33:52]

[ok] - https://www.turbochargerpros.com/buynow/40-39212_gg
[尝试次数：8] - [剩余数量：335] - [当前时间：10:33:52]

[ok] - https://www.turbochargerpros.com/buynow/40-31569_gg
[尝试次数：1] - [剩余数量：334] - [当前时间：10:33:54]

[ok] - https://www.turbochargerpros.com/buynow/40-35700_gg
[尝试次数：2] - [剩余数量：333] - [当前时间：10:33:54]

[ok] - https://www.turbochargerpros.com/buynow/40-30998_gg
[尝试次数：7] - [剩余数量：332] - [当前时间：10:33:55]

[ok] - https://www.turbochargerpros.com/buynow/40-31403_gg
[尝试次数：13] - [剩余数量：331] - [当前时间：1

[ok] - https://www.turbochargerpros.com/buynow/40-30365_gg
[尝试次数：2] - [剩余数量：257] - [当前时间：10:34:18]

[ok] - https://www.turbochargerpros.com/buynow/40-30557_gg
[尝试次数：2] - [剩余数量：256] - [当前时间：10:34:18]

[ok] - https://www.turbochargerpros.com/buynow/40-31587_gg
[尝试次数：8] - [剩余数量：255] - [当前时间：10:34:19]

[ok] - https://www.turbochargerpros.com/buynow/40-30297_gr
[尝试次数：2] - [剩余数量：254] - [当前时间：10:34:19]

[ok] - https://www.turbochargerpros.com/buynow/40-30293_gr
[尝试次数：2] - [剩余数量：253] - [当前时间：10:34:19]

[ok] - https://www.turbochargerpros.com/buynow/40-30297_gg
[尝试次数：3] - [剩余数量：252] - [当前时间：10:34:19]

[ok] - https://www.turbochargerpros.com/buynow/40-30807_gg
[尝试次数：7] - [剩余数量：251] - [当前时间：10:34:20]

[ok] - https://www.turbochargerpros.com/buynow/40-30298_gg
[尝试次数：6] - [剩余数量：250] - [当前时间：10:34:20]

[ok] - https://www.turbochargerpros.com/buynow/40-30296_gr
[尝试次数：5] - [剩余数量：249] - [当前时间：10:34:20]

[ok] - https://www.turbochargerpros.com/buynow/40-30292_gr
[尝试次数：1] - [剩余数量：248] - [当前时间：10:34:21]



[ok] - https://www.turbochargerpros.com/buynow/40-35660_gg
[尝试次数：6] - [剩余数量：175] - [当前时间：10:34:44]

[ok] - https://www.turbochargerpros.com/buynow/40-35646_gg
[尝试次数：2] - [剩余数量：174] - [当前时间：10:34:45]

[ok] - https://www.turbochargerpros.com/buynow/40-35647_gg
[尝试次数：1] - [剩余数量：173] - [当前时间：10:34:45]

[ok] - https://www.turbochargerpros.com/buynow/40-35654_gg
[尝试次数：1] - [剩余数量：172] - [当前时间：10:34:45]

[ok] - https://www.turbochargerpros.com/buynow/40-35651_gr
[尝试次数：1] - [剩余数量：171] - [当前时间：10:34:45]

[ok] - https://www.turbochargerpros.com/buynow/40-35664_gr
[尝试次数：2] - [剩余数量：170] - [当前时间：10:34:45]

[ok] - https://www.turbochargerpros.com/buynow/40-35645_gr
[尝试次数：2] - [剩余数量：169] - [当前时间：10:34:46]

[ok] - https://www.turbochargerpros.com/buynow/40-35643_gr
[尝试次数：1] - [剩余数量：168] - [当前时间：10:34:46]

[ok] - https://www.turbochargerpros.com/buynow/40-35675_gg
[尝试次数：12] - [剩余数量：167] - [当前时间：10:34:46]

[ok] - https://www.turbochargerpros.com/buynow/40-35642_gr
[尝试次数：1] - [剩余数量：166] - [当前时间：10:34:46]


[ok] - https://www.turbochargerpros.com/buynow/40-31138_gg
[尝试次数：4] - [剩余数量：92] - [当前时间：10:35:16]

[ok] - https://www.turbochargerpros.com/buynow/40-31727_gg
[尝试次数：4] - [剩余数量：91] - [当前时间：10:35:16]

[ok] - https://www.turbochargerpros.com/buynow/40-35614_gg
[尝试次数：3] - [剩余数量：90] - [当前时间：10:35:17]

[ok] - https://www.turbochargerpros.com/buynow/40-31150_s1
[尝试次数：3] - [剩余数量：89] - [当前时间：10:35:17]

[ok] - https://www.turbochargerpros.com/buynow/40-30979_gg
[尝试次数：1] - [剩余数量：88] - [当前时间：10:35:17]

[ok] - https://www.turbochargerpros.com/buynow/40-30942_gg
[尝试次数：1] - [剩余数量：87] - [当前时间：10:35:17]

[ok] - https://www.turbochargerpros.com/buynow/40-30933_gr
[尝试次数：3] - [剩余数量：86] - [当前时间：10:35:18]

[ok] - https://www.turbochargerpros.com/buynow/40-31410_gg
[尝试次数：5] - [剩余数量：85] - [当前时间：10:35:18]

[ok] - https://www.turbochargerpros.com/buynow/40-30819_gg
[尝试次数：1] - [剩余数量：84] - [当前时间：10:35:19]

[ok] - https://www.turbochargerpros.com/buynow/40-30934_gr
[尝试次数：3] - [剩余数量：83] - [当前时间：10:35:19]

[ok] - htt

[ok] - https://www.turbochargerpros.com/buynow/40-30398_gg
[尝试次数：1] - [剩余数量：9] - [当前时间：10:35:52]

[ok] - https://www.turbochargerpros.com/buynow/40-30190_gr
[尝试次数：1] - [剩余数量：8] - [当前时间：10:35:52]

[ok] - https://www.turbochargerpros.com/buynow/40-30163_gr
[尝试次数：4] - [剩余数量：7] - [当前时间：10:35:53]

[ok] - https://www.turbochargerpros.com/buynow/40-30119_gg
[尝试次数：3] - [剩余数量：6] - [当前时间：10:35:53]

[ok] - https://www.turbochargerpros.com/buynow/40-30205_gg
[尝试次数：9] - [剩余数量：5] - [当前时间：10:35:54]

[ok] - https://www.turbochargerpros.com/buynow/40-30109_gg
[尝试次数：2] - [剩余数量：4] - [当前时间：10:35:54]

[ok] - https://www.turbochargerpros.com/buynow/40-30129_gg
[尝试次数：2] - [剩余数量：3] - [当前时间：10:35:54]

[ok] - https://www.turbochargerpros.com/buynow/40-30104_gg
[尝试次数：1] - [剩余数量：2] - [当前时间：10:35:54]

[ok] - https://www.turbochargerpros.com/buynow/40-30173_gg
[尝试次数：5] - [剩余数量：1] - [当前时间：10:35:55]

[ok] - https://www.turbochargerpros.com/buynow/40-30095_gg
[尝试次数：1] - [剩余数量：0] - [当前时间：10:35:55]

[ok] - https://www.t